# Disaster Tweets

### Import data

In [13]:
# proper imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ml imports
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle 
from sklearn.metrics import accuracy_score, recall_score, precision_score

# HF imports
from datasets import Dataset, DatasetDict, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
from sentence_transformers import SentenceTransformer, util

random_seed = 42

In [21]:
# import train as df
df = pd.read_csv('train.csv')
print('Columns:',df.columns)
print('Shape of the dataframe:', df.shape, '\n')

#checking out nan values in text columns
print(df['id'].isnull().sum())
print(df['keyword'].isnull().sum())
print(df['location'].isnull().sum())
print(df['text'].isnull().sum())

Columns: Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Shape of the dataframe: (7613, 5) 

0
61
2533
0


In [27]:
# look at rows where keyword is not nan
df[df['keyword'].notnull()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [16]:
# shuffle data
df = shuffle(df,random_state=random_seed)

# split data into train and val
df_train, df_val = train_test_split(df, test_size=0.20, stratify=df['target']) 

# distributions of target
print("train: \n",df_train['target'].value_counts(), "\n")
print("val: \n",df_val['target'].value_counts())

#drop columns that are not needed

train: 
 0    3473
1    2617
Name: target, dtype: int64 

val: 
 0    869
1    654
Name: target, dtype: int64


In [19]:
# convert dataframes to HuggingFace dataset dictionary
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset = DatasetDict({"train": dataset_train, "val": dataset_val})

# rename index and labels column
dataset = dataset.rename_columns({'__index_level_0__': 'idx', 'target': 'labels'})

In [28]:
checkpoint = 'distilroberta-base'

# load pretrained tokenizer (checkpoint must match with model)
print(f"Checkpoint: {checkpoint}")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
     # no padding, handled by data collator in dynamic fashion
     return tokenizer(examples["text"], truncation=True)

# tokenize data
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# data collator to deal with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer)

Checkpoint: distilroberta-base


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [29]:
# set seed for reproducability
set_seed(random_seed)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    num_labels=2, 
    problem_type='single_label_classification'
    )

# hyperparams 
training_args = TrainingArguments(
    output_dir="junk_filter_model_1",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    #learning_rate=5e-6, # set to 5e-6 for roberta-large to avoid local optimum
    load_best_model_at_end=True,
    metric_for_best_model='precision',
    seed=random_seed
    )

# adding more metrics to the trainer
metrics = {}
for metric in ['accuracy','precision', 'recall']:
    metrics[metric] = load_metric(metric,'mrpc')

def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels)["precision"]
    recall = metric2.compute(predictions=predictions, references=labels)["recall"]
    accuracy = metric3.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall}

# define a trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics,
)

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

In [30]:
# launch trainer
train_result = trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, id, text, keyword, location. If idx, id, text, keyword, location are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/Users/ericjohansson/opt/miniconda3/envs/junk-filter-env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6090
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1524


  0%|          | 0/1524 [00:00<?, ?it/s]

KeyboardInterrupt: 